# Notebook for cleaning census data

In [36]:
import pandas as pd
import os 
import sys
import requests
import json
import io
import matplotlib as plt
%pylab inline

# User defined function that reads in API key. See folder for sample file. 
from censusAPI import myAPI

Populating the interactive namespace from numpy and matplotlib


/Applications/anaconda/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [37]:
url = "https://api.census.gov/data/2016/acs/acs5/variables.json"
resp = requests.request('GET', url)
aff1y = json.loads(resp.text)

In [38]:
affkeys = np.array(list(aff1y['variables'].keys()))

## Total Population

#### B01001_001E --> Estimate Total Population

In [81]:
pop_var = 'B05001_001E'
word = "Pop"
url = "https://api.census.gov/data/2016/acs/acs5?get=NAME," + pop_var +\
"&for=zip%20code%20tabulation%20area:*&key=" + myAPI

resp = requests.request('GET', url)
aff1y = json.loads(resp.text)
resp = requests.request('GET', url).content

resp = requests.request('GET', url).content
pop = pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']','')),
                    dtype={'zip code tabulation area': 'str'})

In [82]:
pop.head()

,NAME,B05001_001E,zip code tabulation area,Unnamed: 3
0,ZCTA5 01001,17423,01001,NaN
1,ZCTA5 01002,29970,01002,NaN
2,ZCTA5 01003,11296,01003,NaN
3,ZCTA5 01005,5228,01005,NaN
4,ZCTA5 01007,14888,01007,NaN


In [83]:
(pop['zip code tabulation area']).shape

(33120,)

In [84]:
pop[pop['zip code tabulation area'] == '10001']

,NAME,B05001_001E,zip code tabulation area,Unnamed: 3
2427,ZCTA5 10001,23332,10001,NaN


In [85]:
pop = pop.drop("Unnamed: 3", axis=1)
pop.columns = ['Name','total','zipcode']
pop.head()

,Name,total,zipcode
0,ZCTA5 01001,17423,01001
1,ZCTA5 01002,29970,01002
2,ZCTA5 01003,11296,01003
3,ZCTA5 01005,5228,01005
4,ZCTA5 01007,14888,01007


In [97]:
pop.dtypes

Name       object
total      object
zipcode    object
dtype: object

### For Citizenship (Citizen / NonCitizen)

##### B05001_001E --> total

##### B05001_006E --> non citizen

In [44]:
variable = 'B05001_001E'  
word = 'citizen_pop'
url = "https://api.census.gov/data/2016/acs/acs5?get=NAME," + variable +\
"&for=zip%20code%20tabulation%20area:*&key=" + myAPI

resp = requests.request('GET', url)
aff1y = json.loads(resp.text)
resp = requests.request('GET', url).content
citizen_pop = pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']','')), dtype={'zip code tabulation area': 'str'})
print(citizen_pop.shape)
citizen_pop = citizen_pop.rename(columns={variable : word})
citizen_pop = citizen_pop[citizen_pop['zip code tabulation area'].str.startswith('1')]
citizen_pop.drop('Unnamed: 3', axis=1, inplace=True)
citizen_pop.head()

(33120, 4)


,NAME,citizen_pop,zip code tabulation area
2427,ZCTA5 10001,23332,10001
2428,ZCTA5 10002,78096,10002
2429,ZCTA5 10003,56767,10003
2430,ZCTA5 10004,3044,10004
2431,ZCTA5 10005,8710,10005


#### Non-Citizen

In [45]:
variable = 'B05001_006E'  
word = 'foreign_pop'
url = "https://api.census.gov/data/2016/acs/acs5?get=NAME," + variable +\
"&for=zip%20code%20tabulation%20area:*&key=" + myAPI

resp = requests.request('GET', url)
aff1y = json.loads(resp.text)
resp = requests.request('GET', url).content
for_pop = pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']','')), dtype={'zip code tabulation area': 'str'})
print(for_pop.shape)
for_pop = for_pop.rename(columns={variable : word})
for_pop = for_pop[for_pop['zip code tabulation area'].str.startswith('1')]
for_pop.drop('Unnamed: 3', axis=1, inplace=True)
print(for_pop.shape)
for_pop.head()

(33120, 4)
(3658, 3)


,NAME,foreign_pop,zip code tabulation area
2427,ZCTA5 10001,3787,10001
2428,ZCTA5 10002,12731,10002
2429,ZCTA5 10003,7075,10003
2430,ZCTA5 10004,462,10004
2431,ZCTA5 10005,1412,10005


In [65]:
pop_df = pd.merge(citizen_pop, for_pop[['NAME', 'foreign_pop']])
pop_df = pop_df.drop('NAME', axis=1)
pop_df.columns = ['citizen_pop','zipcode','foreign_pop']
pop_df = pop_df.set_index('zipcode')
pop_df = pop_df.astype(float)
pop_df['prctForeign'] = pop_df['foreign_pop'] / pop_df['citizen_pop'] *100

print(pop_df.shape)
pop_df.head()

(3658, 3)


,citizen_pop,foreign_pop,prctForeign
zipcode,,,
10001,23332.0,3787.0,16.230927
10002,78096.0,12731.0,16.301731
10003,56767.0,7075.0,12.463227
10004,3044.0,462.0,15.177398
10005,8710.0,1412.0,16.211251


In [109]:
pop_df.dtypes

citizen_pop    float64
foreign_pop    float64
prctForeign    float64
dtype: object

## Insurance

##### B27002_001E --> Private Insurance
##### B27003_001E --> Public Insurance

### Total Population

In [47]:
variable = 'S0101_C01_001E'  
word = 'total_pop'
url = "https://api.census.gov/data/2016/acs/acs5/subject?get=NAME," + variable +\
"&for=zip%20code%20tabulation%20area:*&key=" + myAPI

resp = requests.request('GET', url)
aff1y = json.loads(resp.text)
resp = requests.request('GET', url).content
total_pop = pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']','')), dtype={'zip code tabulation area': 'str'})
print(total_pop.shape)
total_pop = total_pop.rename(columns={variable : word})
total_pop = total_pop[total_pop['zip code tabulation area'].str.startswith('1')]
total_pop.drop('Unnamed: 3', axis=1, inplace=True)
total_pop.head()

(33120, 4)


,NAME,total_pop,zip code tabulation area
2427,ZCTA5 10001,23332,10001
2428,ZCTA5 10002,78096,10002
2429,ZCTA5 10003,56767,10003
2430,ZCTA5 10004,3044,10004
2431,ZCTA5 10005,8710,10005


In [108]:
total_pop.dtypes

NAME                        object
total_pop                    int64
zip code tabulation area    object
dtype: object

### Insured

In [48]:
variable = 'S2701_C02_001E'  
word = 'insured'
url = "https://api.census.gov/data/2016/acs/acs5/subject?get=NAME," + variable +\
"&for=zip%20code%20tabulation%20area:*&key=" + myAPI

resp = requests.request('GET', url)
aff1y = json.loads(resp.text)
resp = requests.request('GET', url).content
insured = pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']','')), dtype={'zip code tabulation area': 'str'})
print(insured.shape)
insured = insured.rename(columns={variable : word})
insured = insured[insured['zip code tabulation area'].str.startswith('1')]
insured.drop('Unnamed: 3', axis=1, inplace=True)
insured.head()

(33120, 4)


,NAME,insured,zip code tabulation area
2427,ZCTA5 10001,21859,10001
2428,ZCTA5 10002,71629,10002
2429,ZCTA5 10003,53737,10003
2430,ZCTA5 10004,2854,10004
2431,ZCTA5 10005,8202,10005


### Uninsured

In [49]:
variable = 'S2701_C04_001E'  
word = 'uninsured'
url = "https://api.census.gov/data/2016/acs/acs5/subject?get=NAME," + variable +\
"&for=zip%20code%20tabulation%20area:*&key=" + myAPI

resp = requests.request('GET', url)
aff1y = json.loads(resp.text)
resp = requests.request('GET', url).content
uninsured = pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']','')), dtype={'zip code tabulation area': 'str'})
print(uninsured.shape)
uninsured = uninsured.rename(columns={variable : word})
uninsured = uninsured[uninsured['zip code tabulation area'].str.startswith('1')]
uninsured.drop('Unnamed: 3', axis=1, inplace=True)
uninsured.head()

(33120, 4)


,NAME,uninsured,zip code tabulation area
2427,ZCTA5 10001,1473,10001
2428,ZCTA5 10002,6149,10002
2429,ZCTA5 10003,2876,10003
2430,ZCTA5 10004,190,10004
2431,ZCTA5 10005,503,10005


In [50]:
insurance = pd.merge(insured, uninsured[['NAME', 'uninsured']])
insurance = insurance.merge(total_pop[['NAME', 'total_pop']], on='NAME')
insurance['prctInsured'] = (insurance['insured'] / insurance['total_pop']) * 100
insurance['prctUninsured'] = (insurance['uninsured'] / insurance['total_pop']) * 100
#insurance['test'] = insurance['prctInsured'] + insurance['prctUninsured']
insurance.columns = ['Name','insured','zipcode','uninsured','totalPop','%Insured',
                    '%Uninsured']
print(insurance.shape)
insurance.head()

(3658, 7)


,Name,insured,zipcode,uninsured,totalPop,%Insured,%Uninsured
0,ZCTA5 10001,21859,10001,1473,23332,93.686782,6.313218
1,ZCTA5 10002,71629,10002,6149,78096,91.719166,7.873643
2,ZCTA5 10003,53737,10003,2876,56767,94.662392,5.066324
3,ZCTA5 10004,2854,10004,190,3044,93.758213,6.241787
4,ZCTA5 10005,8202,10005,503,8710,94.167623,5.774971


In [107]:
insurance.dtypes

Name           object
insured         int64
zipcode        object
uninsured       int64
totalPop        int64
%Insured      float64
%Uninsured    float64
dtype: object

## Vehicle Access

In [51]:
variable = 'B08014_002E'  
word = 'vehicle_none'
url = "https://api.census.gov/data/2016/acs/acs5?get=NAME," + variable +\
"&for=zip%20code%20tabulation%20area:*&key=" + myAPI

resp = requests.request('GET', url)
aff1y = json.loads(resp.text)
resp = requests.request('GET', url).content
vehicle_none = pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']','')), dtype={'zip code tabulation area': 'str'})
print(vehicle_none.shape)
vehicle_none = vehicle_none.rename(columns={variable : word})
vehicle_none = vehicle_none[vehicle_none['zip code tabulation area'].str.startswith('1')]
vehicle_none.drop('Unnamed: 3', axis=1, inplace=True)
vehicle_none.head()

(33120, 4)


,NAME,vehicle_none,zip code tabulation area
2427,ZCTA5 10001,10584,10001
2428,ZCTA5 10002,25969,10002
2429,ZCTA5 10003,24664,10003
2430,ZCTA5 10004,1567,10004
2431,ZCTA5 10005,5692,10005


In [52]:
variable = 'B08014_001E'  
word = 'vehicle_totalpop'
url = "https://api.census.gov/data/2016/acs/acs5?get=NAME," + variable +\
"&for=zip%20code%20tabulation%20area:*&key=" + myAPI

resp = requests.request('GET', url)
aff1y = json.loads(resp.text)
resp = requests.request('GET', url).content
vehicle_totalpop = pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']','')), dtype={'zip code tabulation area': 'str'})
print(vehicle_totalpop.shape)
vehicle_totalpop = vehicle_totalpop.rename(columns={variable : word})
vehicle_totalpop = vehicle_totalpop[vehicle_totalpop['zip code tabulation area'].str.startswith('1')]
vehicle_totalpop.drop('Unnamed: 3', axis=1, inplace=True)
vehicle_totalpop.head()

(33120, 4)


,NAME,vehicle_totalpop,zip code tabulation area
2427,ZCTA5 10001,13655,10001
2428,ZCTA5 10002,33780,10002
2429,ZCTA5 10003,31944,10003
2430,ZCTA5 10004,2048,10004
2431,ZCTA5 10005,6692,10005


In [71]:
vehicle = pd.merge(vehicle_none, vehicle_totalpop, on ="zip code tabulation area")
vehicle = vehicle.drop("NAME_y", axis = 1)
vehicle.columns = ['Name','vehicle_none','zipcode','vehicle_totalpop']
vehicle['%vehicle'] = (vehicle['vehicle_none'] / vehicle['vehicle_totalpop']) *100
vehicle = vehicle.drop(["Name", 'vehicle_none', 'vehicle_totalpop'], axis = 1)
vehicle.head()

,zipcode,%vehicle
0,10001,77.510070
1,10002,76.876850
2,10003,77.210118
3,10004,76.513672
4,10005,85.056784


In [106]:
vehicle.dtypes

zipcode      object
%vehicle    float64
dtype: object

## Food Stamps

##### S2201_C01_001E	

In [125]:
fst_var = 'S2201_C01_001E'
word = "Pop"
url = "https://api.census.gov/data/2016/acs/acs5/subject?get=NAME," + fst_var +\
"&for=zip%20code%20tabulation%20area:*&key=" + myAPI

resp = requests.request('GET', url)
aff1y = json.loads(resp.text)
resp = requests.request('GET', url).content

resp = requests.request('GET', url).content
foodstamp = pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']','')),
                    dtype={'zip code tabulation area': 'str'})

foodstamp = foodstamp[foodstamp['zip code tabulation area'].str.startswith('1')]
foodstamp.drop("Unnamed: 3", axis=1, inplace = True)
foodstamp.columns = ['Name','foodStamp','zipcode']
print(foodstamp.shape)
print(foodstamp.dtypes)
foodstamp.head()

(3658, 3)
Name         object
foodStamp     int64
zipcode      object
dtype: object


,Name,foodStamp,zipcode
2427,ZCTA5 10001,11967,10001
2428,ZCTA5 10002,33327,10002
2429,ZCTA5 10003,27258,10003
2430,ZCTA5 10004,1592,10004
2431,ZCTA5 10005,4295,10005


In [126]:
foodstamp.zipcode = foodstamp.zipcode.astype(str)
foodstamp = pd.merge(foodstamp, pop, on ="zipcode")
foodstamp.drop(["Name_x", "Name_y"], axis=1, inplace = True)
# foodstamp = pop_df.drop('NAME', axis=1)
foodstamp.columns = ['foodStamp','zipcode','total']
foodstamp = foodstamp.set_index('zipcode')
foodstamp = foodstamp.astype(float)
foodstamp['%foodStamp'] = (foodstamp['foodStamp'] / foodstamp['total']) *100
# foodstamp = pop_df.drop(['foodStamp', 'total'], axis=1)

print(foodstamp.shape)
foodstamp.head()

(3658, 3)


,foodStamp,total,%foodStamp
zipcode,,,
10001,11967.0,23332.0,51.290074
10002,33327.0,78096.0,42.674401
10003,27258.0,56767.0,48.017334
10004,1592.0,3044.0,52.299606
10005,4295.0,8710.0,49.311137


In [120]:
foodstamp.zipcode = foodstamp.zipcode.astype(str)
foodstamp.dtypes

Name_x       object
foodStamp     int64
zipcode      object
Name_y       object
total        object
dtype: object

## Poverty

##### S0601_C01_048E

In [139]:
pov_var = 'S0601_C01_049E'
word = "Pop"
url = "https://api.census.gov/data/2016/acs/acs5/subject?get=NAME," + pov_var +\
"&for=zip%20code%20tabulation%20area:*&key=" + myAPI

resp = requests.request('GET', url)
aff1y = json.loads(resp.text)
resp = requests.request('GET', url).content

resp = requests.request('GET', url).content
poverty = pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']','')), 
                     dtype={'zip code tabulation area': 'str'})
print(poverty.shape)
poverty = poverty[poverty['zip code tabulation area'].str.startswith('1')]
poverty.drop("Unnamed: 3", axis=1, inplace = True)
poverty.columns = ['Name','poverty','zipcode']
poverty.head()

(33120, 4)


,Name,poverty,zipcode
2427,ZCTA5 10001,17.5,10001
2428,ZCTA5 10002,27.8,10002
2429,ZCTA5 10003,9.9,10003
2430,ZCTA5 10004,4.8,10004
2431,ZCTA5 10005,12.1,10005


In [140]:
poverty.poverty.max()

'94.4'

In [141]:
# poverty = pd.merge(poverty, pop, on ="zipcode")
poverty.drop(["Name"], axis=1, inplace = True)
# poverty = poverty.set_index('zipcode')
poverty.columns = ['%poverty', 'zipcode']
# poverty = poverty.astype(float)
# poverty['%poverty'] = (poverty['poverty'] / poverty['total']) *100

In [142]:
print(poverty.shape)
print(poverty.dtypes)
poverty['%poverty'] = poverty['%poverty'].astype(float)
# poverty = poverty.set_index('zipcode')
poverty.head()

(3658, 2)
%poverty    object
zipcode     object
dtype: object


,%poverty,zipcode
2427,17.5,10001
2428,27.8,10002
2429,9.9,10003
2430,4.8,10004
2431,12.1,10005


## Urban / Rural area

In [60]:
urban = pd.read_csv("zip_urban2.csv")
urban = urban[['UANAME', 'ZCTA5']]
urban.columns = ['UrbanAreaAssos', 'zipcode']
urban.head(3)

,UrbanAreaAssos,zipcode
0,"Abbeville, LA Urban Cluster",70510
1,"Abbeville, LA Urban Cluster",70528
2,"Abbeville, LA Urban Cluster",70533


In [61]:
urban['urban/rural'] = np.where(urban.UrbanAreaAssos == 'Not in a 2010 urban area', 0, 1)
urban.head(3)

,UrbanAreaAssos,zipcode,urban/rural
0,"Abbeville, LA Urban Cluster",70510,1
1,"Abbeville, LA Urban Cluster",70528,1
2,"Abbeville, LA Urban Cluster",70533,1


In [146]:
urban.zipcode = urban.zipcode.astype(str)
urban.dtypes

UrbanAreaAssos    object
zipcode           object
urban/rural        int64
dtype: object

----

# Merge all the datasets

In [148]:
df = pop.merge(pop_df, left_on = 'zipcode', right_index=True)
df = df.merge(insurance, on='zipcode')
df = df.merge(vehicle, on='zipcode')
df = df.merge(foodstamp, left_on='zipcode', right_index=True)
df = df.merge(poverty, on='zipcode')
df = df.merge(urban, on='zipcode')

df.drop(["Name_x", "Name_y", 'total_x', 'total_y', 'citizen_pop', 'foreign_pop', 'insured', 'uninsured', 'totalPop',
        '%Insured', 'foodStamp', 'UrbanAreaAssos'], axis=1, inplace = True)
df = df.set_index('zipcode')
df.head()

,prctForeign,%Uninsured,%vehicle,%foodStamp,%poverty,urban/rural
zipcode,,,,,,
10001,16.230927,6.313218,77.510070,51.290074,17.5,1
10002,16.301731,7.873643,76.876850,42.674401,27.8,1
10003,12.463227,5.066324,77.210118,48.017334,9.9,1
10004,15.177398,6.241787,76.513672,52.299606,4.8,1
10005,16.211251,5.774971,85.056784,49.311137,12.1,1


In [149]:
df.to_csv('dataACS.csv')